## Scrape the content

In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
result=requests.get("https://transcripts.fandom.com/wiki/Avengers:_Infinity_War")

In [3]:
src=result.content
soup=BeautifulSoup(src,'lxml')

In [4]:
import pandas as pd

In [5]:
import re

In [6]:
import re
import pandas as pd
s = []
text = []
for link in soup.find_all('div'):
    if ("id","mw-content-text") in link.attrs.items():
        for l in link.find_all('p'):
            if l.find("b"):
                author = re.findall(r'(.*?):', l.text)
                text = re.sub(r'\[.*?\]',"",l.text)
                text = re.sub(r'.*?:',"", text)
                text = re.sub(r'\n','',text)
                try:
                    s.append((author[0],text))
                except:
                    pass

In [7]:
import numpy as np
pd.set_option('display.max_colwidth',-1)
r = np.random.randint(100)

#data=pd.DataFrame(s, columns = ['Title', 'Text'])[r:(r+10)]
data=pd.DataFrame(s, columns = ['Title', 'Text'])

In [8]:
data2=data.copy()

In [9]:
data2.head()

,Title,Text
0,Asgardian PA,"This is the Asgardian refugee vessel Statesman. We are under assault, I repeat, we are under assault - The engines are dead, life support failing. Requesting aid from any vessel within range. We are 22 jump points out of Asgard. Our crew is made up of Asgardian families, we have very few soldiers here. This is not a warcraft. I repeat, this is not a warcraft!"
1,Ebony Maw,"Hear me, and rejoice. You have had the privilege of being saved by the Great Titan.... You may think this is suffering... no. it is salvation. Universal scales tip toward balance because of your sacrifice. Smile... for even in death, you have become Children of Thanos."
2,Thanos,"I know what it's like to lose. To feel so desperately that you're right... yet to fail, nonetheless. It's frightening. Turns the legs to jelly. I ask you, to what end? Dread it. Run from it. Destiny arrives all the same. And now, it's here. Or should I say... I AM."
3,Thor,You talk too much.
4,Thanos,"The Tesseract, or your brother's head. I assume you have a preference."


## Using Spacy 

In [10]:
import spacy

In [11]:
nlp=spacy.load("en_core_web_md")

In [12]:
text=list(data2['Text'].str.lower())

In [13]:
text[:2]

[' this is the asgardian refugee vessel statesman. we are under assault, i repeat, we are under assault - the engines are dead, life support failing. requesting aid from any vessel within range. we are 22 jump points out of asgard.\xa0 our crew is made up of asgardian families, we have very few soldiers here. this is not a warcraft.\xa0i repeat, this is not a warcraft!',
 ' hear me, and rejoice. you have had the privilege of being saved by the great titan.... you may think this is suffering... no. it is salvation. universal scales tip\xa0toward balance because of your sacrifice. smile...  for even in death, you have become children of thanos.']

In [14]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline

### Tokenize the Data

In [15]:
import string
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

In [16]:
## Create our list of punctuation marks

In [17]:
punc=string.punctuation

In [18]:
punc=punc + '...' + '....'+''+'--'

In [19]:
## Create our list of stopwords

In [20]:
stop_words = spacy.lang.en.stop_words.STOP_WORDS

In [21]:
type(stop_words)

set

In [1]:
#stop_words = stop_words.union(set(stop_new))

In [ ]:
## Load the English parser

In [24]:
parser=English()

In [25]:
text[1]

' hear me, and rejoice. you have had the privilege of being saved by the great titan.... you may think this is suffering... no. it is salvation. universal scales tip\xa0toward balance because of your sacrifice. smile...  for even in death, you have become children of thanos.'

In [ ]:
## Creating our tokenizer function

In [ ]:
def spacy_tokenizer(sentence):
    # Creating our token object, which is used to create documents with linguistic annotations.
    mytokens=parser(sentence)
    
    ## Lemmatize
    mytokens=[word.lemma_.lower().strip() if word.lemma_!="-PRON-" else word.lower_ for word in mytokens]
    
    ## Remove stop words
    mytokens=[word for word in mytokens if word not in stop_words and word not in punc]
    
    ## return list of tokens
    return mytokens
    

In [ ]:
text_copy=text.copy()

In [ ]:
tokens=[]
for i in text:
    tokens.append(spacy_tokenizer(i))

In [ ]:
tokens[:5]

In [ ]:
len(tokens)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline

In [ ]:
from nltk.probability import FreqDist

In [ ]:
tokens[:1]

In [ ]:
flat_list = []
for sublist in tokens:
    for item in sublist:
        flat_list.append(item)

In [ ]:
flat_list[:2]

In [ ]:
fdist = FreqDist(flat_list)

In [ ]:
from matplotlib import rcParams
rcParams['figure.figsize'] = 15,3
fdist.plot(50)
plt.show()

In [ ]:
#stop_new=['stone','know','yes','come','...','think','right','time']

In [ ]:
final_text=[]
for i in tokens:
    new=[]
    for j in i:
        new.append(j)
    final_text.append(' '.join(new))

In [ ]:
final_text_ = []
for i in final_text:
    final_text_.append(re.sub('\d+',"", i))
final_text_.append(i)
final_text_

## LSA

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_df = 0.5, stop_words = 'english', max_features= 1000, smooth_idf=True)

In [ ]:
X = vectorizer.fit_transform(final_text_)

In [ ]:
X.shape

### Truncated_SVD

In [ ]:
from sklearn.decomposition import TruncatedSVD

# SVD represent documents and terms in vectors
# Since we only have 170 features, take n_components = 100 (should be less than features)

svd_model = TruncatedSVD(n_components=100, algorithm='randomized', n_iter=100, random_state=123)

svd_model.fit(X)
svd_model.transform(X).shape
#svd_model.components_.shape

In [ ]:
# This are the weightages of individual terms in per topic / doc:
(svd_model.components_[0])

In [ ]:
#The components of svd_model are our topics, and we can access them using svd_model.components_.
#Finally, let’s print a few most important words in each of the 21 topics and see how our model has done.

terms = vectorizer.get_feature_names()

for i, weight in enumerate(svd_model.components_):
    terms_weight = zip(terms, weight)
    #print(i)  i is the topic and svd_model has weight of each term in that topic
    #print(list(terms_comp))
    sorted_terms = sorted(terms_weight, key= lambda x:x[1], reverse=True)[:7]
    print("Topic "+str(i)+": ")
    y = []
    for t in sorted_terms:
        y.append(t[0])
    print(y)

## LDA Model for Topic Modelling

### CountVectorizer

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
final_text[:2]

In [ ]:
vectorizer = CountVectorizer()
X_lda = vectorizer.fit_transform(final_text_)

In [ ]:
X_lda.toarray()

In [ ]:
vectorizer.get_feature_names()

In [ ]:
data_lda=pd.DataFrame(X_lda.toarray(),columns=vectorizer.get_feature_names())

In [ ]:
data_lda.head()

In [ ]:
from gensim import matutils,models
import scipy.sparse

In [ ]:
# One of the required inputs is a term-document matrix
tdm = data_lda.transpose()
tdm.head()

In [ ]:
# We're going to put the term-document matrix into a new gensim format, from df --> sparse matrix --> gensim corpus
sparse_counts = scipy.sparse.csr_matrix(tdm)
corpus = matutils.Sparse2Corpus(sparse_counts)

In [ ]:
sparse_counts.toarray()

In [ ]:
corpus

In [ ]:
# Gensim also requires dictionary of the all terms and 
#their respective location in the term-document matrix


cv = vectorizer
id2word = dict((v, k) for k, v in cv.vocabulary_.items()) ## we want location:term dictionary

In [ ]:
id2word

In [ ]:
# Now that we have the corpus (term-document matrix) and id2word (dictionary of location: term),
# we need to specify two other parameters as well - 
#the number of topics and the number of passes

lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=2, 
                      passes=10)
lda.print_topics()

In [ ]:
# LDA for num_topics = 3
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=3, passes=10)
lda.print_topics()

## Analysis of Verbs, Nouns and Adjectives

In [ ]:
text[:2]

### Get the verb Count:

In [ ]:
for i in text:
    doc1=nlp(i)
    for i in doc1:
        #print(i.pos_)
        if i.is_stop:
            continue
        if i.pos_ == 'VERB':
            if i.lemma_ in pos_count:
                pos_count[i.lemma_] += 1
            else:
                pos_count[i.lemma_] = 1    

In [ ]:
print("top 10 VERBs {}".format(sorted(pos_count.items(), key=lambda kv: kv[1], reverse=True)[:10]))    

In [ ]:
pos_count_verb={}
pos_count_verb=sorted(pos_count.items(), key=lambda kv: kv[1], reverse=True)[:10]

In [ ]:
def Convert(tup, di): 
    di=dict(tup)
    return di 

In [ ]:
dictionary={}
pos_count_verb=Convert(pos_count_verb, dictionary)

In [ ]:
pos_count_verb

In [ ]:
### Distribution of Verbs:

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.bar(range(len(pos_count_verb)), list(pos_count_verb.values()), align='center')
plt.xticks(range(len(pos_count_verb)), list(pos_count_verb.keys()))
plt.show()

### Get the Adverbs:

In [ ]:
adverb_count={}

In [ ]:

for i in text:
    doc1=nlp(i)
    for i in doc1:
        #print(i.pos_)
        if i.is_stop:
            continue
        if i.pos_ == 'ADV':
                try:
                    if i.lemma_ in adverb_count:
                        adverb_count[i.lemma_] += 1
                    else:
                        adverb_count[i.lemma_] = 1 
                except:
                    print(i.lemma_)

In [ ]:
print("top 10 ADVERBs {}".format(sorted(adverb_count.items(), key=lambda kv: kv[1], reverse=True)[:10]))    

In [ ]:
adverb_count_new={}
adverb_count_new=sorted(adverb_count.items(), key=lambda kv: kv[1], reverse=True)[:10]
def Convert(tup, di): 
    di=dict(tup)
    return di 

In [ ]:
dictionary={}
adverb_count=Convert(adverb_count_new, dictionary)

In [ ]:
adverb_count

In [ ]:
plt.figure(figsize=(10,7))
plt.bar(range(len(adverb_count)), list(adverb_count.values()), align='center')
plt.xticks(range(len(adverb_count)), list(adverb_count.keys()))
plt.show()

### Get the Nouns

In [ ]:
noun_count={}

In [ ]:

for i in text:
    doc1=nlp(i)
    for i in doc1:
        #print(i.pos_)
        if i.is_stop:
            continue
        if i.pos_ == 'NOUN':
                try:
                    if i.lemma_ in noun_count:
                        noun_count[i.lemma_] += 1
                    else:
                        noun_count[i.lemma_] = 1 
                except:
                    print(i.lemma_)

In [ ]:
print("top 10 Nouns {}".format(sorted(noun_count.items(), key=lambda kv: kv[1], reverse=True)[:10]))    

In [ ]:
noun_count_new={}
noun_count_new=sorted(noun_count.items(), key=lambda kv: kv[1], reverse=True)[:10]
def Convert(tup, di): 
    di=dict(tup)
    return di 

In [ ]:
dictionary={}
noun_count=Convert(noun_count_new, dictionary)

In [ ]:
plt.figure(figsize=(10,7))
plt.bar(range(len(noun_count)), list(noun_count.values()), align='center')
plt.xticks(range(len(noun_count)), list(noun_count.keys()))
plt.show()

### Get the Adjectives

In [ ]:
adjective_count={}

In [ ]:

for i in text:
    doc1=nlp(i)
    for i in doc1:
        #print(i.pos_)
        if i.is_stop:
            continue
        if i.pos_ == 'ADJ':
                try:
                    if i.lemma_ in adjective_count:
                        adjective_count[i.lemma_] += 1
                    else:
                        adjective_count[i.lemma_] = 1 
                except:
                    print(i.lemma_)

In [ ]:
print("top 10 Adjectives {}".format(sorted(adjective_count.items(), key=lambda kv: kv[1], reverse=True)[:10]))    

In [ ]:
adjective_count_new={}
adjective_count_new=sorted(adjective_count.items(), key=lambda kv: kv[1], reverse=True)[:10]
def Convert(tup, di): 
    di=dict(tup)
    return di 

In [ ]:
dictionary={}
adjective_count=Convert(adjective_count_new, dictionary)

In [ ]:
plt.figure(figsize=(10,7))
plt.bar(range(len(adjective_count)), list(adjective_count.values()), align='center')
plt.xticks(range(len(adjective_count)), list(adjective_count.keys()))
plt.show()

## Entity wise Verbs, Nouns

### Top 30 Entities

In [ ]:
entities = {}
for i in text:
    doc1=nlp(i)
# named entities
    for ent in doc1.ents:
        #if(ent.label_ =='PERSON'):
    # Print the entity text and its label
        if ent.text in entities:
            entities[ent.text] += 1
        else:
            entities[ent.text] = 1
        
print("top entities {}".format(sorted(entities.items(),
      key=lambda kv: kv[1], reverse=True)[:50]))

In [ ]:
entities_new={}
entities_new=sorted(entities.items(), key=lambda kv: kv[1], reverse=True)[:30]
def Convert(tup, di): 
    di=dict(tup)
    return di 

In [ ]:
dictionary={}
entities=Convert(entities_new, dictionary)

In [ ]:
plt.figure(figsize=(40,20))
plt.bar(range(len(entities)), list(entities.values()), align='center')
plt.xticks(range(len(entities)), list(entities.keys()))
plt.show()

### Entity wise Verb

#### Verbs by Thanos

In [23]:
thanos_verb={}
for j in text:
    doc1=nlp(j)
    for ent in doc1:
        if(ent.text=='Thanos'):
            for i in doc1:
                #print(i.pos_)
                if i.is_stop:
                    continue
                if i.pos_ == 'VERB':
                    if i.lemma_ in thanos_verb:
                        thanos_verb[i.lemma_] += 1
                    else:
                        thanos_verb[i.lemma_] = 1    

In [165]:
print("top 10 Verbs {}".format(sorted(thanos_verb.items(), key=lambda kv: kv[1], reverse=True)[:10]))    

top 10 Verbs [('know', 8), ('kill', 7), ('go', 6), ('get', 5), ('stop', 4), ('come', 3), ('need', 3), ('talk', 2), ('steal', 2), ('see', 2)]


#### Nouns by Thanos

In [163]:
thanos_noun={}
for i in text:
    doc1=nlp(i)
    for ent in doc1.ents:
        if(ent.text=='Thanos'):
            for i in doc1:
                #print(i.pos_)
                if i.is_stop:
                    continue
                if i.pos_ == 'NOUN':
                    if i.lemma_ in thanos_noun:
                        thanos_noun[i.lemma_] += 1
                    else:
                        thanos_noun[i.lemma_] = 1    

In [164]:
print("top 10 Nouns {}".format(sorted(thanos_noun.items(), key=lambda kv: kv[1], reverse=True)[:10]))    

top 10 Adjectives [('stone', 5), ('daughter', 3), ('way', 3), ('experience', 2), ('ship', 2), ('moron', 2), ('army', 2), ('life', 2), ('head', 2), ('year', 2)]


### Tony Stark Verbs

In [166]:
Stark_verb={}
for i in text:
    doc1=nlp(i)
    for ent in doc1.ents:
        if(ent.text=='Stark'):
            for i in doc1:
                #print(i.pos_)
                if i.is_stop:
                    continue
                if i.pos_ == 'VERB':
                    if i.lemma_ in Stark_verb:
                        Stark_verb[i.lemma_] += 1
                    else:
                        Stark_verb[i.lemma_] = 1    

In [167]:
print("top 10 Verbs {}".format(sorted(Stark_verb.items(), key=lambda kv: kv[1], reverse=True)[:10]))    

top 10 Verbs [('wanna', 4), ('know', 2), ('beam', 1), ('smell', 1), ('give', 1), ('understand', 1), ('come', 1), ('save', 1), ('hesitate', 1), ('let', 1)]


In [170]:
Stark_noun={}
for i in text:
    doc1=nlp(i)
    for ent in doc1.ents:
        if(ent.text=='Tony'):
            for i in doc1:
                #print(i.pos_)
                if i.is_stop:
                    continue
                if i.pos_ == 'NOUN':
                    if i.lemma_ in Stark_noun:
                        Stark_noun[i.lemma_] += 1
                    else:
                        Stark_noun[i.lemma_] = 1    

In [171]:
print("top 10 Verbs {}".format(sorted(Stark_noun.items(), key=lambda kv: kv[1], reverse=True)[:10]))    

top 10 Verbs [('plague', 1), ('planet', 1), ('population', 1), ('attack', 1), ('telling', 1), ('creature', 1), ('universe', 1), ('hand', 1), ('bot', 1), ('choice', 1)]


In [172]:
Groot_noun={}
for i in text:
    doc1=nlp(i)
    for ent in doc1.ents:
        if(ent.text=='Groot'):
            for i in doc1:
                #print(i.pos_)
                if i.is_stop:
                    continue
                if i.pos_ == 'NOUN':
                    if i.lemma_ in Groot_noun:
                        Groot_noun[i.lemma_] += 1
                    else:
                        Groot_noun[i.lemma_] = 1    

In [173]:
print("top 10 Nouns {}".format(sorted(Groot_noun.items(), key=lambda kv: kv[1], reverse=True)[:10]))    

top 10 Nouns [('face', 2), ('thing', 2), ('moron', 2), ('ship', 1), ('assortment', 1), ('pirate', 1), ('angel', 1), ('way', 1), ('captain', 1), ('game', 1)]
